In [1]:
import json
import numpy as np

In [2]:
root_folder = '/home/lyt/code/concept_vqa'
data_folder = '{}/dataTVQA'.format(root_folder)
tvqa_folder = '/home/data3/lyt/TextVQA'

In [3]:
#=======================================

In [4]:
split_name = 'test'
jsdata = json.load(open('{}/TextVQA_0.5_{}.json'.format(tvqa_folder, split_name),'r'))
print(jsdata.keys())

dict_keys(['dataset_type', 'dataset_name', 'dataset_version', 'data'])


In [5]:
for k in jsdata.keys():
    if k != 'data':
        print('{}: {}'.format(k,jsdata[k]))

dataset_type: test
dataset_name: textvqa
dataset_version: 0.5


In [6]:
data = jsdata['data']
print(len(data),type(data))

5734 <class 'list'>


In [7]:
pair = data[3]
pair.keys()

dict_keys(['question', 'image_id', 'image_classes', 'flickr_original_url', 'flickr_300k_url', 'image_width', 'image_height', 'ocr_tokens', 'ocr_info', 'question_tokens', 'question_id', 'set_name'])

In [8]:
for k in pair.keys():
    if k != 'ocr_info':
        print('{}: {}'.format(k,pair[k]))

question: who makes the phone?
image_id: a644c4c46d7c0b0f
image_classes: ['Laptop', 'Mobile phone']
flickr_original_url: https://c8.staticflickr.com/3/2490/4082192766_947a9ba3f8_o.jpg
flickr_300k_url: https://c2.staticflickr.com/3/2490/4082192766_54d583035d_z.jpg
image_width: 1024
image_height: 768
ocr_tokens: ['MOTOROLA', 'ווון', '김']
question_tokens: ['who', 'makes', 'the', 'phone']
question_id: 39605
set_name: test


In [9]:
#------save im_id:ocr_information------

In [10]:
qid_ocr = {}
for pair in data:
    qid = pair['question_id']
    if qid not in qid_ocr:
        qid_ocr[qid] = [token.lower() for token in pair['ocr_tokens'] if token != '']
json.dump(qid_ocr, open('{}/qid_ocr_{}.json'.format(data_folder, split_name),'w'))
len(qid_ocr)

5734

In [11]:
qid_ocr[39602]

['nokia',
 'e71',
 'noperator',
 '3006.2000',
 '12)',
 'lnร',
 'busanes',
 'report',
 'મ.mal()',
 'eeling',
 'network',
 'fวอed',
 '*4ে',
 '()']